<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class02/class02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autograd

In [1]:
import torch
import torch.nn as nn

In [2]:
# grad = True
tensor = torch.rand(3, requires_grad=True)
print(tensor)
y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)
print(tensor.grad) # 2*tensor + 12

tensor([0.7063, 0.1640, 0.4938], requires_grad=True)
tensor([6.7063, 6.1640, 6.4938], grad_fn=<AddBackward0>)
tensor([136.9232, 115.9849, 128.5074], grad_fn=<AddBackward0>)
tensor(127.1385, grad_fn=<MeanBackward0>)
tensor([13.4126, 12.3280, 12.9876])


In [3]:
# grad = False
tensor = torch.rand(3, requires_grad=False)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)

tensor([0.4755, 0.3705, 0.3704])
tensor([6.4755, 6.3705, 6.3704])
tensor([127.7960, 123.7496, 123.7474])
tensor(125.0977)


RuntimeError: ignored

In [4]:
# not scalar
tensor = torch.tensor([0.37, 0.58, 0.33], requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z.backward()

tensor([0.3700, 0.5800, 0.3300], requires_grad=True)
tensor([6.3700, 6.5800, 6.3300], grad_fn=<AddBackward0>)
tensor([123.7307, 131.8892, 122.2067], grad_fn=<AddBackward0>)


RuntimeError: ignored

In [5]:
# grad 
tensor = torch.rand(3, requires_grad = True)
print(tensor)
y = tensor.detach()
print(y)

tensor([0.0023, 0.5288, 0.9770], requires_grad=True)
tensor([0.0023, 0.5288, 0.9770])


In [6]:
# grad 
with torch.no_grad():
  y = tensor + 2
  print(y)

tensor([2.0023, 2.5288, 2.9770])


In [7]:
# accumulated gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])
tensor([12., 12., 12., 12.])
tensor([15., 15., 15., 15.])


In [8]:
# empty gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


## toy example

In [9]:
# toy example
x = torch.tensor([[1,-1], [2,3], [5,2]], dtype=torch.float32) # 3x2
y = torch.tensor([[1],[0],[1]], dtype=torch.float32)

w1 = torch.rand(2,3, requires_grad=True)
w2 = torch.rand(3,3, requires_grad=True)
w3 = torch.rand(3,2, requires_grad=True)
w4 = torch.rand(2,1, requires_grad=True)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
bce = nn.BCELoss()

def forward(inputs):
    inputs = torch.matmul(inputs, w1)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w2)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w3)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w4)
    outputs = sigmoid(inputs)
    return outputs

# loss
def loss(y_true, y_pred):
    return bce(y_pred, y_true)


learning_rate = 0.01
epochs = 1000

for epoch in range(epochs):
    # forward pass
    y_hat = forward(inputs=x)

    # loss
    bce_loss = loss(y_true=y, y_pred=y_hat)

    # backward loss
    bce_loss.backward()

    # update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w3 -= learning_rate * w3.grad
        w4 -= learning_rate * w4.grad

    # zero gradients
    w1.grad.zero_()
    w2.grad.zero_()
    w3.grad.zero_()
    w4.grad.zero_()

    if epoch % 5 == 0:
        print(f"epoch {epoch + 1}: \nw1 = {w1}\n w2 = {w2}\n w3 = {w3}\n w4 = {w4}, loss = {bce_loss:.8f}")

epoch 1: 
w1 = tensor([[0.9952, 0.5675, 0.7258],
        [0.5906, 0.5716, 0.5766]], requires_grad=True)
 w2 = tensor([[0.2165, 0.9708, 0.7192],
        [0.9301, 0.3639, 0.3747],
        [0.3643, 0.1264, 0.3396]], requires_grad=True)
 w3 = tensor([[0.2827, 0.5013],
        [0.2794, 0.9302],
        [0.0647, 0.1140]], requires_grad=True)
 w4 = tensor([[0.3160],
        [0.1839]], requires_grad=True), loss = 1.10560644
epoch 6: 
w1 = tensor([[0.9907, 0.5616, 0.7242],
        [0.5775, 0.5618, 0.5719]], requires_grad=True)
 w2 = tensor([[0.2094, 0.9608, 0.7177],
        [0.9242, 0.3557, 0.3735],
        [0.3580, 0.1176, 0.3382]], requires_grad=True)
 w3 = tensor([[0.2648, 0.4926],
        [0.2605, 0.9210],
        [0.0464, 0.1051]], requires_grad=True)
 w4 = tensor([[0.2791],
        [0.0895]], requires_grad=True), loss = 0.82743686
epoch 11: 
w1 = tensor([[0.9898, 0.5606, 0.7239],
        [0.5710, 0.5561, 0.5695]], requires_grad=True)
 w2 = tensor([[0.2065, 0.9571, 0.7172],
        [0.9216

# Dataset & DataLoader

In [11]:
from torch.utils.data import Dataset, DataLoader

In [18]:
class ScoreDataset(Dataset):
  def __init__(self):
    self.x = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15],[16,17,18]]) # 6x3
    self.y = torch.tensor([[1],[0],[1],[1],[1],[0]]) # 6x1

  def __getitem__(self,index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.x.size(0)

In [21]:
score_dataset = ScoreDataset()

In [22]:
print(score_dataset.x)
print(score_dataset.y)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]])
tensor([[1],
        [0],
        [1],
        [1],
        [1],
        [0]])


In [26]:
dataLoader = DataLoader(dataset=score_dataset,batch_size=2,shuffle=True)

In [27]:
for i, (x,y) in enumerate(dataLoader):
  print(f"{i}: x={x}, y={y}")

0: x=tensor([[10, 11, 12],
        [16, 17, 18]]), y=tensor([[1],
        [0]])
1: x=tensor([[7, 8, 9],
        [4, 5, 6]]), y=tensor([[1],
        [0]])
2: x=tensor([[13, 14, 15],
        [ 1,  2,  3]]), y=tensor([[1],
        [1]])
